In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as st

#**1. Анализ исходных данных**

#*Посмотрим на исходные данные*

In [ ]:
df = pd.read_csv('new_dataframe.csv')

Поля:

● User_id - id пользователя

● Hour - час старта сессии

● App_opened - приложение открыто

● Price_seen - пользователь ввел данные маршрута и показана цена

● Order_made - пользователь кликнул по кнопке заказа

● Surge - в этот период был включен surge (надбавка к стоимости поездки в период повышенного спроса)

● Ride_completed - поездка успешно завершена

● User_cancelled - пользователь отменил поездку

● Age - возраст пользователя

● City_center_order - заказ был сделан из центра города

● Distance - дистанция в км

● Rfm - rfm-сегмент пользователя

In [ ]:
df.head()

,Unnamed: 0,user_id,hour,os,order_class,surge,app_opened,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance,age,rfm
0,0,867689,12,iOS,business,no surge,1,1,1,1,0,0,7.982135,20,low
1,1,752172,5,Android,economy,no surge,1,1,1,1,0,1,2.908468,27,high
2,2,486559,15,Android,comfort,no surge,1,1,1,1,0,0,7.224614,21,high
3,3,304024,0,Android,economy,no surge,1,1,1,1,0,1,1.874349,52,low
4,4,139420,0,Android,business,no surge,1,1,1,1,0,0,10.704778,19,low


#*Поищем проблемы и неточности в данных*

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101500 entries, 0 to 101499
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         101500 non-null  int64  
 1   user_id            101500 non-null  int64  
 2   hour               101500 non-null  int64  
 3   os                 101500 non-null  object 
 4   order_class        101500 non-null  object 
 5   surge              91431 non-null   object 
 6   app_opened         101500 non-null  int64  
 7   price_seen         101500 non-null  int64  
 8   order_made         101500 non-null  int64  
 9   ride_completed     101500 non-null  int64  
 10  user_cancelled     101500 non-null  int64  
 11  city_center_order  101500 non-null  int64  
 12  distance           91431 non-null   float64
 13  age                101500 non-null  int64  
 14  rfm                101500 non-null  object 
dtypes: float64(1), int64(10), object(4)
memory usage: 1

In [ ]:
df.describe()

,Unnamed: 0,user_id,hour,app_opened,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance,age
count,101500.000000,101500.000000,101500.000000,101500.0,101500.000000,101500.000000,101500.000000,101500.000000,101500.000000,91431.000000,101500.000000
mean,50749.500000,549874.802207,11.481429,1.0,0.900798,0.731389,0.620365,0.111025,0.571675,5.371152,25.906384
std,29300.670499,258600.104176,6.917522,0.0,0.298934,0.443239,0.485299,0.314164,0.494838,4.118531,7.838328
min,0.000000,100093.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.010946,18.000000
25%,25374.750000,328223.000000,5.000000,1.0,1.000000,0.000000,0.000000,0.000000,0.000000,2.438335,20.000000
50%,50749.500000,548827.000000,11.000000,1.0,1.000000,1.000000,1.000000,0.000000,1.000000,4.279565,24.000000
75%,76124.250000,773051.000000,17.000000,1.0,1.000000,1.000000,1.000000,0.000000,1.000000,7.129814,29.000000
max,101499.000000,999978.000000,23.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,40.268966,69.000000


Кажется, что с данными все в порядке

#*Инсайты*

Взглянув на данные, можем заметить некоторые особенности:

● В период Surge видим более низкую price-to-order конверсию

● Для устройств Android наблюдаем более высокие юзерские отмены

● Если Заказ из центра, то видим более низкие юзерские отмены

● Если Comfort & Business классы, то более низкие юзерские отмены, более длинные поездки

#**2.Формирование гипотез**

#*Сформируем некоторые гипотезы*

1. Если мы перестанем добавлять Surge к economy-классу, то мы увеличим конверсию price-to-order на 5%

2. Если мы повысим цену на 5% для заказов из центра, то мы увеличим выручку, не увеличивая количества юзерских отмен

3. Если мы увеличим цену на 15% для comfort & Business классов, то увеличим нашу выручку, не увеличивая количества юзерских отмен

4. Если мы снизим цену поездки для пользователей на Android на 5%, то мы сократим долю юзерских отмен на 2%

#*Проведем эксперсс-анализ гипотез по методу ICE*

In [ ]:
hypotheses = pd.read_csv('Hypotheses.csv')

In [ ]:
hypotheses

,Гипотеза,Impact,Confidence,На чем основан,Effort,ICE,Unnamed: 6
0,Если мы перестанем добавлять Surge к \neconomy...,20 000 $,40 %,data,5 000 $,3 000 $,4
1,Если мы повысим цену на 5% для заказов из \nце...,30 000 $,60 %,data,5 000 $,13 000 $,3
2,Если мы увеличим цену на 15% для \ncomfort & B...,50 000 $,50 %,data,5 000 $,20 000 $,2
3,Если мы снизим цену поездки для \nпользователе...,60 000 $,80 %,data,5 000 $,43 000 $,1


#*Выбираем гипотезу для теста*

Выбираем гипотезу: 

**"Если мы снизим цену поездки для пользователей на Android на 5%, то мы сократим долю юзерских отмен на 2%".** 

Эта гипотеза была выбрана как самая перспективная в ходе проведения ICE - анализа

Параметры гипотезы:

Тестируемое изменение: как изменится доля юзерских отмен для пользователей на Android

Бизнес-цель: уменьшение доли отмененных поездок

Генеральная совокупность: все пользователи на Android

Ключевая метрика: юзерские отмены(user_cancelled)

Вторичная метрика: общая выручка

Уровень значимости установим на отметке 5%

Соответственно уровень достоверности (Confidence Level) = 95%

#**3. Подготовка к А/В-тестированию**

#*Формируем выборку*

##*Определяем размер выборки*
уровень достоверности = 95%

доверительный интервал = 3%

генеральная совокупность - пока под вопросом, вычислим ее

In [ ]:
df = df[df['os']=='Android']
df.groupby('os').count()

,Unnamed: 0,user_id,hour,order_class,surge,app_opened,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance,age,rfm
os,,,,,,,,,,,,,,
Android,56696,56696,56696,56696,50049,56696,56696,56696,56696,56696,56696,50049,56696,56696


генеральная совокупность - 56696

С помощью сторонних кулькуляторов определим размер выборки

**Необходимый размер выборки равен 1047**

В общем-то, можно чуть увеличить выборку, до 1500 пользователей

##*Проведем сплитование*

Для начала, выберем 1500 случайных пользователей

In [ ]:
android = df.sample(1500)
android

,Unnamed: 0,user_id,hour,os,order_class,surge,app_opened,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance,age,rfm
82891,82891,857438,23,Android,economy,no surge,1,1,1,0,1,1,1.120134,20,low
17474,17474,463510,9,Android,comfort,surge,1,1,0,0,0,0,1.784340,18,high
44992,44992,342273,17,Android,comfort,no surge,1,1,1,1,0,0,5.887042,33,low
2104,2104,667749,15,Android,comfort,no surge,1,1,1,1,0,0,4.257710,23,low
53259,53259,376846,22,Android,comfort,no surge,1,1,1,1,0,1,5.651919,18,high
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83425,83425,926294,23,Android,economy,no surge,1,1,1,0,1,1,5.171167,40,low
98015,98015,643575,21,Android,comfort,surge,1,1,1,1,0,1,1.031309,40,high
96447,96447,443350,1,Android,economy,no surge,1,1,0,0,0,1,4.380426,32,high
99023,99023,231658,13,Android,economy,no surge,1,1,1,1,0,1,5.597911,19,low


Теперь разделим на две случайные выборки

In [ ]:
android["group"] = np.random.randint(0, 2, android.shape[0])
android

,Unnamed: 0,user_id,hour,os,order_class,surge,app_opened,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance,age,rfm,group
82891,82891,857438,23,Android,economy,no surge,1,1,1,0,1,1,1.120134,20,low,1
17474,17474,463510,9,Android,comfort,surge,1,1,0,0,0,0,1.784340,18,high,1
44992,44992,342273,17,Android,comfort,no surge,1,1,1,1,0,0,5.887042,33,low,1
2104,2104,667749,15,Android,comfort,no surge,1,1,1,1,0,0,4.257710,23,low,1
53259,53259,376846,22,Android,comfort,no surge,1,1,1,1,0,1,5.651919,18,high,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83425,83425,926294,23,Android,economy,no surge,1,1,1,0,1,1,5.171167,40,low,0
98015,98015,643575,21,Android,comfort,surge,1,1,1,1,0,1,1.031309,40,high,1
96447,96447,443350,1,Android,economy,no surge,1,1,0,0,0,1,4.380426,32,high,1
99023,99023,231658,13,Android,economy,no surge,1,1,1,1,0,1,5.597911,19,low,0


Контрольная и тестовая группы

In [ ]:
control_group = android[android["group"]==0]
control_group

,Unnamed: 0,user_id,hour,os,order_class,surge,app_opened,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance,age,rfm,group
37657,37657,659118,20,Android,economy,no surge,1,1,1,1,0,1,2.592485,32,low,0
22509,22509,385716,22,Android,economy,surge,1,1,1,0,1,1,3.164926,18,low,0
52215,52215,459948,19,Android,business,surge,1,1,1,1,0,0,7.060056,21,low,0
80883,80883,886893,10,Android,comfort,surge,1,1,1,1,0,1,5.077740,23,high,0
22553,22553,960775,22,Android,economy,no surge,1,1,1,1,0,1,1.595145,31,low,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55800,55800,837685,13,Android,business,no surge,1,1,1,1,0,0,16.255012,46,high,0
67475,67475,166795,5,Android,comfort,no surge,1,1,0,0,0,0,8.359177,30,high,0
83425,83425,926294,23,Android,economy,no surge,1,1,1,0,1,1,5.171167,40,low,0
99023,99023,231658,13,Android,economy,no surge,1,1,1,1,0,1,5.597911,19,low,0


In [ ]:
control_group = control_group.loc[:,["user_cancelled"]]
control_group

,user_cancelled
37657,0
22509,1
52215,0
80883,0
22553,0
...,...
55800,0
67475,0
83425,1
99023,0


In [ ]:
test_group = android[android["group"]==1]
test_group

,Unnamed: 0,user_id,hour,os,order_class,surge,app_opened,price_seen,order_made,ride_completed,user_cancelled,city_center_order,distance,age,rfm,group
82891,82891,857438,23,Android,economy,no surge,1,1,1,0,1,1,1.120134,20,low,1
17474,17474,463510,9,Android,comfort,surge,1,1,0,0,0,0,1.784340,18,high,1
44992,44992,342273,17,Android,comfort,no surge,1,1,1,1,0,0,5.887042,33,low,1
2104,2104,667749,15,Android,comfort,no surge,1,1,1,1,0,0,4.257710,23,low,1
53259,53259,376846,22,Android,comfort,no surge,1,1,1,1,0,1,5.651919,18,high,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57028,57028,501368,8,Android,economy,surge,1,1,1,1,0,0,5.490930,27,high,1
96445,96445,354069,13,Android,economy,surge,1,1,0,0,0,1,1.321817,24,low,1
82577,82577,776077,17,Android,comfort,no surge,1,1,1,1,0,1,0.957060,18,low,1
98015,98015,643575,21,Android,comfort,surge,1,1,1,1,0,1,1.031309,40,high,1


In [ ]:
test_group = test_group.loc[:,["user_cancelled"]]
test_group

,user_cancelled
82891,1
17474,0
44992,0
2104,0
53259,0
...,...
57028,0
96445,0
82577,0
98015,0


#*Проверяем выборки на однородность*

Прежде чем использовать t-тест, необходимо проверить равенство дисперсий. Сделаем это с помощью F-теста. Используем собстенную функцию.

H0 - дисперсии выборок одинаковы

Н1 - дисперсии выборок не одинаковы, выборки имеют статистически значимые различия

In [ ]:
def f_test(group1, group2):
    f = np.var(group1, ddof=1)/np.var(group2, ddof=1)
    nun = group1.size-1
    dun = group2.size-1
    p_value = 1-st.f.cdf(f, nun, dun)
    return f, p_value

In [ ]:
stat, p = f_test(control_group, test_group)

print('Статистика=%.3f, p=%.5f' % (stat, p))

if p > 0.05:
    print('Не отклоняем нулевую гипотезу, дисперссии, вероятно, одинаковые')
else:
    print('Отклоняем нулевую гипотезу, дисперссии, вероятно, различаются')

Статистика=1.127, p=0.05162
Не отклоняем нулевую гипотезу, дисперссии, вероятно, одинаковые


Проверяем на однородность с помощью двустороннего t-теста Стьюедента

Н0 - выборки не имеют статистически значимых различий

Н1 - выборки имеют статистически значимые различия

In [ ]:
stat, p = st.ttest_ind(control_group,test_group)

print('Статистика=%.3f, p=%.5f' % (stat, p))
if p > 0.05:
    print('Не отклоняем нулевую гипотезу, средние, вероятно, одинаковые')
else:
    print('Отклоняем нулевую гипотезу, средние, вероятно, различаются')

Статистика=1.097, p=0.27285
Не отклоняем нулевую гипотезу, средние, вероятно, одинаковые


Выборки не отличаются, значит мы можем использовать их для проведения АВ-теста

**Посмотрим на долю отмен по группам до теста**

In [ ]:
control_group.value_counts()

user_cancelled
0                 634
1                 110
dtype: int64

In [ ]:
cancelled_control_before = 110/744
cancelled_control_before

0.1478494623655914

In [ ]:
test_group.value_counts()

user_cancelled
0                 659
1                  97
dtype: int64

In [ ]:
cancelled_test_before = 97/756
cancelled_test_before

0.1283068783068783

#**4. Проведение А/В-теста**

Контрольную группу оставляем без изменений, для тестовой группы снижаем цену на 5%

Тест будем проводить 2 недели

По истечении 2-х недель анализируем показатели групп

Смотрим на долю отмененных заказов. 

После теста смотрим на тестовую и контрольные группы

In [ ]:
control_group

,user_cancelled
37657,0
22509,1
52215,0
80883,0
22553,0
...,...
55800,0
67475,0
83425,1
99023,0


In [ ]:
test_group

,user_cancelled
index,
82891,0
17474,0
44992,0
2104,0
53259,0
...,...
57028,0
96445,0
82577,0


**Посмотрим на долю отмен по группам после теста**

In [ ]:
control_group.value_counts()

user_cancelled
0                 634
1                 110
dtype: int64

In [ ]:
cancelled_control = 110/744
cancelled_control

0.1478494623655914

In [ ]:
test_group.value_counts()

user_cancelled
0                 668
1                  88
dtype: int64

In [ ]:
cancelled_test = 88/756

In [ ]:
cancelled_test

0.1164021164021164

Видим: 

доля отмененных заказов в контрольной группе - 14.7%

доля отмененных заказов в тестовой группе - 11.6% 

Мы добились желаемого - снизили долю отмененных заказов на 2%. Но привело ли к этому изменение, или эффект случайный?. Нужно проверить сходство выборок

Проверяем на однородность с помощью двустороннего t-теста Стьюедента

Н0 - выборки не имеют статистически значимых различий

Н1 - выборки имеют статистически значимые различия

In [ ]:
stat, p = st.ttest_ind(control_group,test_group)

print('Статистика=%.3f, p=%.5f' % (stat, p))
if p > 0.05:
    print('Не отклоняем нулевую гипотезу, средние, вероятно, одинаковые')
else:
    print('Отклоняем нулевую гипотезу, средние, вероятно, различаются')

Статистика=1.800, p=0.07210
Не отклоняем нулевую гипотезу, средние, вероятно, одинаковые


#**5. Анализ результатов А/В-теста**

#*Вывод:*

## *Хоть мы и видим снижение доли отмененных заказов в тестовой группе, мы не можем достоверно утверждать, что к этому привело наше изменение, так как результат проверки с помощью статистического теста говорит нам об отсутвии статистически значимых различий между группами. Вполне возможно эффект оказался случайным.*

#*Дальнейшие шаги*

##*1. Мы можем закончить тест и отказаться от изменений*
##*2. Мы можем продолжить тест, но тогда нам нужно предпринимать дополнительные шаги:*
##*- Продлить тест на больший промежуток времени*
##*- Увеличить размер выборки*
##*- Можно снизить уровень значимости*
##*3. Если в дальнейшем тест показывает нужные нам результаты, тогда можем внедрять изменения*